This script takes a segmentation and a table of spots, and adds the information about which cell the spot is in to the spot table. You can optionally remove spots outside of cells / nuclei.

## Input
- segmentation masks in either `npy` or `png` format (name should correspond to the name of the image in the spot file)
- `.csv` file of spots in each of the 2 channels, containg following columns:
    - *img* - image name
    - *channel* channel number the spot belongs to  
    - *x, y* and *z* coordiantes

## Output
- `.csv` file of spots with addtional columns:
    - *cell* - cell number in that particular image (0 means outside of cell)
    - *whole_cell* - marks `False` for all cells touching the image border
- optionally removes spots outside of cells

# Functions and imports

In [4]:
import os
from glob import glob
import pandas as pd
import numpy as np
import re
from skimage.morphology import label
from skimage.segmentation import clear_border
from skimage.io import imread

from utils.spot_analysis import add_cell_info

In [5]:
# assigns each spot to a cell and filters spots.csv for spots in cells
def add_cell_info(masks,path_spots,out,filter=True,mask_ending="_cp_masks"):
    
    df = pd.read_csv(path_spots)
        
    df_list = []
    
    for file in masks:
        # subset spots for spots in current image
        name = file.split("/")[-1].split(".")[0]
        name = name.replace(mask_ending, "")
        name = re.sub(r'_ch\d+', '', name)
        subset_df = df[df['img'].str.contains(name)]
        
        # load mask
        file_type = os.path.splitext(file)[1]
        
        if file_type == ".npy":
            mask = np.load(file,allow_pickle=True).item()['masks']
        elif file_type == ".png":
            mask = imread(file)
        else:
            print("Please input valid segmentation masks (.npy and .png supported).")
        
        # label all cells and remove cells on edges
        labelled_mask = label(mask)
        cleared_mask = clear_border(labelled_mask)

        
    # add cell info to spots
        # for 2d masks
        if len(cleared_mask.shape) == 2:            
            cell = labelled_mask[subset_df['y'].astype(int), subset_df['x'].astype(int)]
            subset_df.insert(1, 'cell', cell)

            # info about whether spot is in cell on border
            spot_in_cleared_mask = cleared_mask[subset_df['y'].astype(int), subset_df['x'].astype(int)] != 0
            subset_df.insert(2, 'whole_cell', spot_in_cleared_mask)

            df_list.append(subset_df)
            
        # for 3d masks
        elif len(cleared_mask.shape) == 3:
            cell = labelled_mask[subset_df['z'].astype(int), subset_df['y'].astype(int), subset_df['x'].astype(int)]
            subset_df.insert(1, 'cell', cell)

            # info about whether spot is in cell touching border
            spot_in_cleared_mask = cleared_mask[subset_df['z'].astype(int), subset_df['y'].astype(int), subset_df['x'].astype(int)] != 0
            subset_df.insert(2, 'whole_cell', spot_in_cleared_mask)

            df_list.append(subset_df)
            
    spots = pd.concat(df_list, ignore_index=True)
    
    # remove all spots without a cell (0)
    if filter == True:
        spots.drop(spots[spots['cell'] == 0].index, inplace=True)
    
    spots.to_csv(out, index=False)

# Add cell info to spots
Optionally set `filter = True` to remove spots outside of cells. 

*NOTE:* The segmentation mask name should correspond to the name of the image in the spot file + a suffix / prefix. You have to specify the correct suffix / prefix in the function call, so that the masks can be matched to the images.

In [6]:
in_path = "/home/stumberger/ep2024/example/" # upper level experiment folder
filter = True # whether to filter out spots outside of cells
mask_ending = "_seg"
spot_file= "merge_shift-corrected.csv"

In [7]:
path_spots = f"{in_path}/detections/{spot_file}" #spots file
masks = glob(f"{in_path}/segmentation/*.npy") #all segmentation masks (.npy and .png supported)
out = f"{in_path}/detections/merge_filtered.csv" # where to save new file

#filter=True - exclude spots outside of nuclei
add_cell_info(masks,path_spots,out,filter=filter,mask_ending=mask_ending)